In [1]:
import ipywidgets as widgets
from IPython.display import display
import plotly.offline as pyoff
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.colors as colors
import plotly.io as pio
pio.renderers.default = "notebook"
import numpy as np

In [2]:
import ipywidgets as widgets
from IPython.display import display
import plotly.offline as pyoff
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.colors as colors
import plotly.io as pio
import numpy as np

pio.renderers.default = "notebook"

class Plots:

    @staticmethod
    def histogram(func):
        func.plot = 1
        return func

    @staticmethod
    def scatter(func):
        func.plot = 0
        return func

    @staticmethod
    def variations(func):
        func.plot = 3
        return func

    @staticmethod
    def updateLiveScatter(fig, y, x):
        if not isinstance(fig, go.FigureWidget):
            fig = go.FigureWidget(fig)
            display(fig)

        xs = [*fig.data[0].x, x] if( isinstance(x,
                                               int) or isinstance(x,
                                               float)) else [*fig.data[0].x, *x]
        ys = [*fig.data[0].y, y] if (isinstance(y,
                                               int) or isinstance(y,
                                               float)) else [*fig.data[0].y, *y]

        # ys.append(y)
        with fig.batch_update():
            scatter = fig.data[0]
            scatter.x = xs
            scatter.y = ys

        return fig

    def _callableSync(obj, update_dict):
        if getattr(obj, "_callable", False):

            args = getattr(obj, "_args", ())
            kwds = getattr(obj, "_kwargs", dict()).copy()

            shared = set(kwds) & set(update_dict)
            for k in shared:
                kwds[k] = update_dict[k]
            return obj(args, **kwds)
        return obj

    def createGraph(graph_parameters, display_figure=True, **kwargs):
        """
        graph_parameters={
                        "traces":trace_list,# trace_list.shape = rows,cols
                        "layout":layout_dict,
                        "fig_type":fig_type_str,
                        "fig_parameters":dict|None
                        "fig_functions":{"function_name":"parameters"}
                        **optional_parameters
                        }

        """
        graph_parameters.update(**kwargs)

        traces = np.array(graph_parameters["traces"], dtype=object)

        functions = graph_parameters.get("functions", None)
        fig_functions = graph_parameters.get("fig_functions", None)
        fig_type = graph_parameters.get("fig_type", None)
        fig_parameters = graph_parameters.get("fig_parameters", dict())
        # print(np.shape(traces)!= tuple())
        subplots = np.shape(traces) != tuple() and np.shape(traces)[0] > 1
        # print(np.shape(traces))
        # subplots = len(dimensions) > 0

        if subplots:
            dimensions = [len(traces), len(traces[0])]

            rows = (dimensions[0:1] or [1])[0]
            cols = (dimensions[1:2] or [1])[0]
            fig = make_subplots(rows=rows, cols=cols, **fig_parameters)

            if fig_type == "Widget":
                # base = graph_parameters.get("base", None)
                # if base is None:
                #     raise ValueError("Widget expects a base")
                container = graph_parameters.get("container", None)
                if container is None:
                    raise ValueError("Widget expects a base")

                fig = go.FigureWidget(fig)

            for i in range(rows):
                for j in range(cols):
                    trace = graph_parameters["traces"][i][j]
                    if trace is not None:
                        if isinstance(trace, list):
                            [
                                fig.add_trace(t, row=i + 1, col=j + 1)
                                for t in trace if t is not None
                            ]
                        else:
                            # print(i,j)

                            # print(trace)
                            fig.add_trace(trace, row=i + 1, col=j + 1)

        else:
            fig = go.Figure(**fig_parameters)
            if graph_parameters["traces"] is not None:

                fig.add_trace(graph_parameters["traces"])

        fig.update_layout(graph_parameters["layout"])

        if fig_functions:
            for k, v in fig_functions.items():
                func = getattr(fig, k)
                func = Plots._callableSync(func, locals())

                func(v, **kwargs)

        if functions:
            for k, v in functions.items():
                func = getattr(fig, k)
                # func = _callableSync(func,locals())
                func(fig, v, **kwargs)
        if display_figure:
            if fig_type == "Widget":
                container = Plots._callableSync(container, locals())(fig)
                display(container)
            else:
                fig.show()
        return fig

    def plotGraph(function_data, **kwargs):
        graph_params = Plots._plotGraph(function_data)
        Plots.createGraph(graph_params)
        return graph_params


    @classmethod
    def _plotGraph(cls,interface, **kwargs):
        model_name = interface.MODEL_NAME
        function_name = interface.FUNCTION_NAME
        parameters = interface["parameters"]
        NTrials = interface["NTrials"]
        data = interface["data"]

        func = interface["function"]
        #    Quick fix for compatibility graphVariational
        kwargs["alpha_name"] = parameters._alias["_alpha"]
        kwargs["beta_name"] = parameters._alias["_beta"]
        kwargs["alpha_range"] = parameters.default_alpha_range
        kwargs["beta_range"] = parameters.default_beta_range
        kwargs["function_name"] = function_name

        # kwargs["parameters"] = parameters

        plot = getattr(func, "plot", None)
        title_layout = dict(title=f"{model_name}: {function_name}")
        if plot is not None:
            method = cls.PLOT_MAPPING.get(plot, None)
            if method is not None:

                title_layout = dict(
                    title=f"{model_name}: {method.__name__} for {function_name}"
                )

                graph_params = method(data, **kwargs)

                graph_params["layout"].update(title_layout)

                return graph_params

    @staticmethod
    def graphVariational(data, **kwargs):

        def getKwargVars(**kwargs):

            alpha_name = kwargs[
                "alpha_name"] if "alpha_name" in kwargs else "alpha"

            beta_name = kwargs["beta_name"] if "beta_name" in kwargs else "beta"

            function_name = kwargs[
                "function_name"] if "function_name" in kwargs else "function"

            alpha_range = kwargs[
                "alpha_range"] if "alpha_range" in kwargs else (0,
                                                                alpha_len - 1)
            beta_range = kwargs["beta_range"] if "beta_range" in kwargs else (
                0, beta_len - 1)
            return alpha_name, beta_name, alpha_range, beta_range, function_name

        dimensions = np.shape(data)
        # parameters=parameters[0][0]
        if dimensions[0] > 2:
            _matrix = data
        else:
            raise

        alpha_len, beta_len = len(_matrix), len(_matrix[0])
        alpha_name, beta_name, alpha_range, beta_range, function_name = getKwargVars(
            **kwargs)

        matrix = np.stack(_matrix.tolist())  # beta × alpha × T

        ymax = np.max(_matrix.tolist())
        X = matrix[0][0].shape[0]

        x = np.arange(X)

        Alpha = np.linspace(*alpha_range, alpha_len)

        Beta = np.linspace(*beta_range, beta_len)

        Xalpha, Yalpha = np.meshgrid(x, Alpha)  # left surface  (beta fixed)
        Xbeta, Ybeta = np.meshgrid(x, Beta)  # right surface (alpha fixed)

        beta_idx, alpha_idx = 0, 0

        alpha_surface = go.Surface(
            z=matrix[beta_idx],
            x=Xalpha,
            y=Yalpha,
            colorscale="Viridis",
            cmin=0,
            cmax=ymax,
            showscale=True,
        )

        beta_surface = go.Surface(
            z=matrix[:, alpha_idx],
            x=Xbeta,
            y=Ybeta,
            colorscale="Viridis",
            cmin=0,
            cmax=ymax,
            showscale=True,
        )

        scatter = go.Scatter(x=x, y=matrix[beta_idx, alpha_idx], mode="lines")

        camera = dict(
            eye=dict(x=-1.8, y=-1.8, z=1.0),
            up=dict(x=0.0, y=0.0, z=1.0),
            center=dict(x=0.0, y=0.0, z=0.0),
        )
        fig_parameters = dict(
            specs=[
                [{
                    "type": "surface"
                }, {
                    "type": "surface"
                }],
                [{
                    "colspan": 2,
                    "type": "xy"
                }, None],
            ],
            vertical_spacing=0.08,
            row_heights=[0.75, 0.25],
        )

        layout = dict(
            width=1400,
            height=850,
            scene=dict(
                xaxis_title="x",
                yaxis_title=f"{alpha_name}",
                zaxis_title=f"{function_name}",
                camera=camera,
            ),
            scene2=dict(
                xaxis_title="x",
                yaxis_title=f"{beta_name}",
                zaxis_title=f"{function_name}",
                camera=camera,
            ),
        )

        def container(fig, **kwargs):


            def _idx(val, grid):
                return int(round((val - grid[0]) / (grid[1] - grid[0])))

            alpha_slider = widgets.FloatSlider(
                value=float(Alpha[alpha_idx]),
                min=float(Alpha.min()),
                max=float(Alpha.max()),
                step=float(Alpha[1] - Alpha[0]),
                description=f"{alpha_name}",
                continuous_update=False,
            )

            beta_slider = widgets.FloatSlider(
                value=float(Beta[beta_idx]),
                min=float(Beta.min()),
                max=float(Beta.max()),
                step=float(Beta[1] - Beta[0]),
                description=f"{beta_name}",
                continuous_update=False,
            )

            def refresh(_=None):
                i = _idx(beta_slider.value, Beta)  # current beta index
                j = _idx(alpha_slider.value, Alpha)  # current alpha index

                with fig.batch_update():
                    fig.data[0].z = matrix[i]

                    fig.data[1].z = matrix[:, j]

                    # 1‑D line
                    fig.data[2].y = matrix[i, j]

                    fig.layout.title.text = (
                        f"{function_name} –  {alpha_name} = {Alpha[j]:.2f}, "
                        f"{function_name} = {Beta[i]:.2f}")

            alpha_slider.observe(refresh, names="value")
            beta_slider.observe(refresh, names="value")
            controls = widgets.VBox([alpha_slider, beta_slider],
                                    layout=widgets.Layout(width="100%"))
            container = widgets.VBox([controls, fig],
                                     layout=widgets.Layout(width="100%"))

            return container

        graph_parameters = {
            "traces": [[alpha_surface, beta_surface], [scatter, None]],
            "layout": layout,
            "fig_type": "Widget",
            "fig_parameters": fig_parameters,
            "container": container,
        }
        return graph_parameters


    @staticmethod
    def graphHistogram(
        data,
        *,
        mode="bar",
        normalise_x_axis=False,
        density=False,
        **kwargs,
    ):

        counts = data[0]
        midpoints = data[1]
        if len(midpoints) == len(counts) + 1:
            # Assume data[1[ is bin edges
            midpoints = (midpoints[:-1] + midpoints[1:]) / 2
            # midpoints = midpoints[:-1]
        if normalise_x_axis:
            if globals().get("Data", None) is None:
                print(
                    "Data module not imported. Please import Data module to use normalise function."
                )
                midpoints = (midpoints - midpoints.min()) / (midpoints.max() -
                                                             midpoints.min())
            else:
                midpoints = Data.normalise(midpoints)
        if density:
            counts = counts / np.sum(counts)
        if mode == "bar":
            trace = go.Bar(x=midpoints, y=counts)
        if mode == "scatter":
            trace = go.Scatter(
                x=midpoints,
                y=counts,
                mode="lines",
                line={"shape": "hv"},
            )

        layout = dict(
            barmode="overlay",
            bargap=0,
        )
        graph_parameters = {
            "traces": trace,
            "layout": layout,
        }
        return graph_parameters

    @staticmethod
    def graphScatter(data, *, normalise_x_axis=False, **kwargs):
        Y = data[0]
        try:
            X = data[1]
        except:
            X = list(range(len(Y)))
        if X is None:
            X = list(range(len(Y)))

        trace = go.Scatter(x=X, y=Y, **kwargs)

        layout = dict(
            barmode="overlay",
            bargap=0,
        )
        graph_parameters = {
            "traces": trace,
            "layout": layout,
        }

        return graph_parameters



    PLOT_MAPPING = {
        0: graphScatter,
        1: NotImplemented,
        2: graphHistogram,
        3: graphVariational,
    }


In [3]:
class Plots:

    DEFAULT_COLORS = colors.DEFAULT_PLOTLY_COLORS
    LEN_DEFAULT_COLORS = len(DEFAULT_COLORS)

    @staticmethod
    def histogram(func):
        func.plot = 1
        return func

    @staticmethod
    def scatter(func):
        func.plot = 0
        return func

    @staticmethod
    def variations(func):
        func.plot = 3
        return func

    @staticmethod
    def updateLiveScatter(fig, y, x):
        if not isinstance(fig, go.FigureWidget):
            fig = go.FigureWidget(fig)
            display(fig)
        if isinstance(x, int) or isinstance(x, float):
            print(x)

        xs = [*fig.data[0].x, x] if isinstance(x, int) or isinstance(
            x, float) else [*fig.data[0].x, *x]
        ys = [*fig.data[0].y, y] if isinstance(y, int) or isinstance(
            y, float) else [*fig.data[0].y, *y]

        # ys.append(y)
        with fig.batch_update():
            scatter = fig.data[0]
            scatter.x = xs
            scatter.y = ys

        return fig
        # fig.show()

    def _callableSync(obj, update_dict):
        if getattr(obj, "_callable", False):

            args = getattr(obj, "_args", ())
            kwds = getattr(obj, "_kwargs", dict()).copy()

            shared = set(kwds) & set(update_dict)
            for k in shared:
                kwds[k] = update_dict[k]
            return obj(args, **kwds)
        return obj

    def createGraph(graph_parameters, display_graph=True, **kwargs):
        """
        graph_parameters={
                        "traces":trace_list,# trace_list.shape = rows,cols
                        "layout":layout_dict,
                        "fig_type":fig_type_str,
                        "fig_parameters":dict|None
                        "fig_functions":{"function_name":"parameters"}
                        **optional_parameters
                        }

        """
        graph_parameters.update(**kwargs)

        traces = np.array(graph_parameters["traces"], dtype=object)

        functions = graph_parameters.get("functions", None)
        fig_functions = graph_parameters.get("fig_functions", None)
        fig_type = graph_parameters.get("fig_type", None)
        fig_parameters = graph_parameters.get("fig_parameters", dict())
        # print(np.shape(traces)!= tuple())
        subplots = np.shape(traces) != tuple() and np.shape(traces)[0] > 1
        # print(np.shape(traces))
        # subplots = len(dimensions) > 0

        if subplots:
            dimensions = [len(traces), len(traces[0])]

            rows = (dimensions[0:1] or [1])[0]
            cols = (dimensions[1:2] or [1])[0]
            fig = Plots.make_subplots(rows=rows, cols=cols, **fig_parameters)

            if fig_type == "Widget":
                # base = graph_parameters.get("base", None)
                # if base is None:
                #     raise ValueError("Widget expects a base")
                container = graph_parameters.get("container", None)
                if container is None:
                    raise ValueError("Widget expects a base")

                fig = go.FigureWidget(fig)

            for i in range(rows):
                for j in range(cols):
                    trace = graph_parameters["traces"][i][j]
                    if trace is not None:
                        if isinstance(trace, list):
                            [
                                fig.add_trace(t, row=i + 1, col=j + 1)
                                for t in trace if t is not None
                            ]
                        else:
                            # print(i,j)

                            # print(trace)
                            fig.add_trace(trace, row=i + 1, col=j + 1)

        else:
            fig = go.Figure(**fig_parameters)
            if graph_parameters["traces"] is not None:

                fig.add_trace(graph_parameters["traces"])

        fig.update_layout(graph_parameters["layout"])

        if fig_functions:
            for k, v in fig_functions.items():
                func = getattr(fig, k)
                func = Plots._callableSync(func, locals())

                func(v, **kwargs)

        if functions:
            for k, v in functions.items():
                func = getattr(fig, k)
                # func = _callableSync(func,locals())
                func(fig, v, **kwargs)
        if display_graph:
            if fig_type == "Widget":
                container = Plots._callableSync(container, locals())(fig)
                display(container)
            else:
                fig.show()
        return fig

    def plotGraph(function_data, **kwargs):
        graph_params = Plots._plotGraph(function_data)
        Plots.createGraph(graph_params)
        return graph_params

    def _plotGraph(interface, **kwargs):
        model_name = interface.MODEL_NAME
        function_name = interface.FUNCTION_NAME
        parameters = interface["parameters"]
        NTrials = interface["NTrials"]
        data = interface["data"]

        func = interface["function"]
        #    Quick fix for compatibility graphVariational
        kwargs["alpha_name"] = parameters._alias["_alpha"]
        kwargs["beta_name"] = parameters._alias["beta_name"]
        kwargs["alpha_range"] = parameters.default_alpha_range
        kwargs["beta_range"] = parameters.default_beta_range
        kwargs["function_name"] = function_name

        # kwargs["parameters"] = parameters

        plot = getattr(func, "plot", None)
        title_layout = dict(title=f"{model_name}: {function_name}")
        if plot is not None:
            method = super().PLOT_MAPPING.get(plot, None)
            if method is not None:

                title_layout = dict(
                    title=f"{model_name}: {method.__name__} for {function_name}"
                )

                graph_params = method(data, **kwargs)

                graph_params["layout"].update(title_layout)

                return graph_params

    @staticmethod
    def graphVariational(data, **kwargs):

        def getKwargVars(**kwargs):

            alpha_name = kwargs[
                "alpha_name"] if "alpha_name" in kwargs else "alpha"

            beta_name = kwargs["beta_name"] if "beta_name" in kwargs else "beta"

            function_name = kwargs[
                "function_name"] if "function_name" in kwargs else "function"

            alpha_range = kwargs[
                "alpha_range"] if "alpha_range" in kwargs else (0,
                                                                alpha_len - 1)
            beta_range = kwargs["beta_range"] if "beta_range" in kwargs else (
                0, beta_len - 1)
            return alpha_name, beta_name, alpha_range, beta_range, function_name

        dimensions = np.shape(data)
        # parameters=parameters[0][0]
        if dimensions[0] > 2:
            _matrix = data
        else:
            raise

        alpha_len, beta_len = len(_matrix), len(_matrix[0])
        alpha_name, beta_name, alpha_range, beta_range, function_name = getKwargVars(
            **kwargs)

        matrix = np.stack(_matrix.tolist())  # beta × alpha × T
        # print(type(matrix))

        ymax = np.max(_matrix.tolist())
        # ymax = np.max(_matrix)
        # print(matrix[0][0].shape)
        # X = matrix.shape[2]
        X = matrix[0][0].shape[0]

        x = np.arange(X)

        # Alpha = np.linspace(alpha_range[0], alpha_range[1], alpha_len)
        Alpha = np.linspace(*alpha_range, alpha_len)

        # Beta = np.linspace(beta_range[0], beta_range[1], beta_len)
        Beta = np.linspace(*beta_range, beta_len)

        Xalpha, Yalpha = np.meshgrid(x, Alpha)  # left surface  (beta fixed)
        Xbeta, Ybeta = np.meshgrid(x, Beta)  # right surface (alpha fixed)

        beta_idx, alpha_idx = 0, 0

        alpha_surface = go.Surface(
            z=matrix[beta_idx],
            x=Xalpha,
            y=Yalpha,
            colorscale="Viridis",
            cmin=0,
            cmax=ymax,
            showscale=True,
        )

        beta_surface = go.Surface(
            z=matrix[:, alpha_idx],
            x=Xbeta,
            y=Ybeta,
            colorscale="Viridis",
            cmin=0,
            cmax=ymax,
            showscale=True,
        )

        scatter = go.Scatter(x=x, y=matrix[beta_idx, alpha_idx], mode="lines")

        camera = dict(
            eye=dict(x=-1.8, y=-1.8, z=1.0),
            up=dict(x=0.0, y=0.0, z=1.0),
            center=dict(x=0.0, y=0.0, z=0.0),
        )
        fig_parameters = dict(
            specs=[
                [{
                    "type": "surface"
                }, {
                    "type": "surface"
                }],
                [{
                    "colspan": 2,
                    "type": "xy"
                }, None],
            ],
            vertical_spacing=0.08,
            row_heights=[0.75, 0.25],
        )

        layout = dict(
            width=1400,
            height=850,
            scene=dict(
                xaxis_title="x",
                yaxis_title=f"{alpha_name}",
                zaxis_title=f"{function_name}",
                camera=camera,
            ),
            scene2=dict(
                xaxis_title="x",
                yaxis_title=f"{beta_name}",
                zaxis_title=f"{function_name}",
                camera=camera,
            ),
        )

        def container(fig, **kwargs):

            # alpha_name, beta_name, func_name = updateSliderNames(**kwargs)

            def _idx(val, grid):
                return int(round((val - grid[0]) / (grid[1] - grid[0])))

            alpha_slider = widgets.FloatSlider(
                value=float(Alpha[alpha_idx]),
                min=float(Alpha.min()),
                max=float(Alpha.max()),
                step=float(Alpha[1] - Alpha[0]),
                description=f"{alpha_name}",
                continuous_update=False,
            )

            beta_slider = widgets.FloatSlider(
                value=float(Beta[beta_idx]),
                min=float(Beta.min()),
                max=float(Beta.max()),
                step=float(Beta[1] - Beta[0]),
                description=f"{beta_name}",
                continuous_update=False,
            )

            def refresh(_=None):
                i = _idx(beta_slider.value, Beta)  # current beta index
                j = _idx(alpha_slider.value, Alpha)  # current alpha index

                with fig.batch_update():
                    fig.data[0].z = matrix[i]

                    fig.data[1].z = matrix[:, j]

                    # 1‑D line
                    fig.data[2].y = matrix[i, j]

                    fig.layout.title.text = (
                        f"{function_name} –  {alpha_name} = {Alpha[j]:.2f}, "
                        f"{function_name} = {Beta[i]:.2f}")

            alpha_slider.observe(refresh, names="value")
            beta_slider.observe(refresh, names="value")
            controls = widgets.VBox([alpha_slider, beta_slider],
                                    layout=widgets.Layout(width="100%"))
            container = widgets.VBox([controls, fig],
                                     layout=widgets.Layout(width="100%"))

            return container

        graph_parameters = {
            "traces": [[alpha_surface, beta_surface], [scatter, None]],
            "layout": layout,
            "fig_type": "Widget",
            "fig_parameters": fig_parameters,
            "container": container,
        }
        return graph_parameters

    @staticmethod
    def graphHistogram(
        data,
        *,
        mode="bar",
        normalise_x_axis=False,
        density=False,
        **kwargs,
    ):

        counts = data[0]
        midpoints = data[1]
        if len(midpoints) == len(counts) + 1:
            # Assume data[1[ is bin edges
            midpoints = (midpoints[:-1] + midpoints[1:]) / 2
            # midpoints = midpoints[:-1]
        if normalise_x_axis:
            if globals().get("Data", None) is None:
                print(
                    "Data module not imported. Please import Data module to use normalise function."
                )
                midpoints = (midpoints - midpoints.min()) / (midpoints.max() -
                                                             midpoints.min())
            else:
                midpoints = Data.normalise(midpoints)
        if density:
            counts = counts / np.sum(counts)
        if mode == "bar":
            trace = go.Bar(x=midpoints, y=counts)
        if mode == "scatter":
            trace = go.Scatter(
                x=midpoints,
                y=counts,
                mode="lines",
                line={"shape": "hv"},
            )

        layout = dict(
            barmode="overlay",
            bargap=0,
        )
        graph_parameters = {
            "traces": trace,
            "layout": layout,
        }
        return graph_parameters

    @staticmethod
    def graphScatter(data, *, normalise_x_axis=False, **kwargs):
        Y = data[0]
        try:
            X = data[1]
        except:
            X = list(range(len(Y)))
        if X is None:
            X = list(range(len(Y)))

        trace = go.Scatter(x=X, y=Y, **kwargs)

        layout = dict(
            barmode="overlay",
            bargap=0,
        )
        graph_parameters = {
            "traces": trace,
            "layout": layout,
        }

        return graph_parameters

    PLOT_MAPPING = {
        0: graphScatter,
        1: NotImplemented,
        2: graphHistogram,
        3: graphVariational,
    }

In [4]:

# import time

# x = []
# y=[]
# data =[x,y]

# trace = Plots.graphScatter(data)
# fig = Plots.createGraph(trace, display=False)


# import time
# for i in range(100):
#     # update(i, np.sin(i / 10))
#     fig=Plots.updateLiveScatter(fig, i, np.sin(i / 10))
#     time.sleep(0.05)

In [ ]:
# Tasks: Read analog signal, plot real-time response, measure peak, decay, duration, and compute statistics

import serial
# from serial import Serial
import time
# import matplotlib.pyplot as plt
import numpy as np

# --- Configuration ---
# PORT = "/dev/tty.usbmodem142201"
PORT = "/dev/cu.usbmodem1101"

BAUDRATE = 9600
DURATION = 30  # seconds
SAMPLING_INTERVAL = 0.001  #4 Hz

arduino = serial.Serial(PORT, 9600)
#while arduino.is_open:
#data = arduino.readline().decode().strip()
#print(f"Sensor Value: {data}")

# --- Data Acquisition ---
timestamps = []
sensor_values = []
start_time = time.time()
trace = Plots.graphScatter([sensor_values, timestamps])
fig = Plots.createGraph(trace, display_figure=False)
print("Reading sensor data...")
while True:
    # print(f"Sensor Value: {arduino.readline()}")
    content= arduino.readline()
    try:
        line = content.decode('utf-8').strip()

        # print(line)

        value = float(line.split()[2])
        current_time = time.time() - start_time
        fig=Plots.updateLiveScatter(fig, value, current_time)
        timestamps.append(current_time)
        sensor_values.append(value)
    except (ValueError,IndexError):
        continue
    # time.sleep(SAMPLING_INTERVAL)

# arduino.close()
# print("Done.")

# # --- Convert to NumPy arrays ---
# timestamps = np.array(timestamps)
# sensor_values = np.array(sensor_values)
# print(timestamps)
# print(sensor_values)

# print(sensor_values)
# print(timestamps)
# # --- Plot sensor response ---
# # plt.figure(figsize=(10, 5))
# # plt.plot(timestamps, sensor_values, label='MQ3 Output')
# # plt.xlabel('Time (s)')
# # plt.ylabel('Sensor Value (ADC)')
# # plt.title('MQ3 Sensor Real-Time Response')
# # plt.legend()
# # plt.grid(True)
# # plt.show()


# # --- Feature Extraction ---
# def analyze_sensor_response(time_data, sensor_data):
#     peak_value = np.max(sensor_data)
#     time_to_peak = time_data[np.argmax(sensor_data)]

#     # Estimate time to return near baseline
#     baseline = np.median(sensor_data[:int(0.1 * len(sensor_data))])
#     threshold = baseline + 0.05 * (peak_value - baseline)
#     decay_time = None
#     for i in range(np.argmax(sensor_data), len(sensor_data)):
#         if sensor_data[i] < threshold:
#             decay_time = time_data[i] - time_to_peak
#             break

#     # Duration above threshold
#     activity_indices = np.where(sensor_data > threshold)[0]
#     duration = (activity_indices[-1] - activity_indices[0]) / (
#         1 / SAMPLING_INTERVAL) if len(activity_indices) > 0 else 0

#     energy = np.trapz(sensor_data, dx=SAMPLING_INTERVAL)

#     return {
#         'peak': peak_value,
#         'time_to_peak': time_to_peak,
#         'decay_time': decay_time,
#         'duration': duration,
#         'energy': energy
#     }


# # --- Compute & Print Results ---
# results = analyze_sensor_response(timestamps, sensor_values)
# print("\nSensor Response Summary:")
# for k, v in results.items():
#     print(f"{k}: {v:.2f}")

# # ---decoding demands---§


# def decode_command(features):
#     """
#     Decodes the sprayer action into one of four commands based on extracted features.
#     You can adjust the thresholds based on your experimental data.
#     """
#     # Example logic (customize as needed):
#     if features['duration'] > 5 and features['energy'] > 10000:
#         return "UP"
#     elif features['duration'] > 2 and features['time_to_peak'] < 2:
#         return "DOWN"
#     elif features['peak'] > 800:
#         return "RIGHT"
#     elif features['decay_time'] and features['decay_time'] > 2:
#         return "LEFT"
#     else:
#         return "UNKNOWN"


# # --- Decode Command ---
# #command = decode_command(results)
# #print(f"\nDecoded Command: {command}")
# ...existing code...


Reading sensor data...


FigureWidget({
    'data': [{'type': 'scatter', 'uid': '70c38bc8-9480-498c-86bc-2f631bd11215', 'x': [], 'y': []}],
    'layout': {'bargap': 0, 'barmode': 'overlay', 'template': '...'}
})

0.00481104850769043
0.06293129920959473
0.06414318084716797
0.06512832641601562
0.06580519676208496
0.06667637825012207
0.06762528419494629
0.06853222846984863
0.06948304176330566
0.07037901878356934
0.07124519348144531
0.07201409339904785
0.07291316986083984
0.07372331619262695
0.07449603080749512
0.07544541358947754
0.07682418823242188
0.07787537574768066
0.07901120185852051
0.07995128631591797
0.08086228370666504
0.08200597763061523
0.08275508880615234
0.08385419845581055
0.0846703052520752
0.08544921875
0.0864102840423584
0.08720827102661133
0.08803606033325195
0.08912825584411621
0.09041833877563477
0.09149837493896484
0.09237027168273926
0.09357619285583496
0.09416604042053223
0.0949242115020752
0.0957801342010498
0.09693121910095215
0.09792518615722656
0.09885621070861816
0.09998011589050293
0.10099196434020996
0.10198807716369629
0.10296010971069336
0.10386133193969727
0.10487818717956543
0.10569429397583008
0.10672926902770996
0.10786819458007812
0.10892415046691895
0.10989117